## ***Emotion Detection from text***

#### RIO-210: Automate detection of different emotions from paragraphs and predict overall emotion

In [1]:
import numpy as np 
import pandas as pd

In [2]:
!ls

'Emotion_Detection(GoogleColab).ipynb'	 emotion.joblib        tfidf1.pkl
'Emotion Detection.ipynb'		 Project_Report.docx
 emotion_detection.py			 text_emotion.csv


In [3]:
# from google.colab import files
# uploaded = files.upload()



---



loading the data & removing irrelevent stuff

In [4]:
data = pd.read_csv("text_emotion.csv")

data.drop("tweet_id",axis=1,inplace=True)
data.drop("author",axis=1,inplace=True)



---




removing empty sentiments

    As no roll of empty sentiments for our model
    
    
    
removing neutral sentiments

    As data corresponging to neutral sentiments have lots of error it's better to remove such data
        for e.g.:
                for content "im saaaaaaaaaaaaaad  i need someone to talk to" 
                sentiment should be sad but it's given neutral 
    

In [5]:
data.drop(data[data["sentiment"] == "empty"].index, inplace=True)
data.drop(data[data["sentiment"] == "neutral"].index, inplace=True)

print(data.shape)
data.head()

(30535, 2)


,sentiment,content
0,happiness,"@havingmysay dude, that is my favorite sandwi..."
2,hate,don't you hate it when you finish all your wor...
3,surprise,mannnn..... @alllyy_ got an iphone!!! im jealo...
4,worry,So i think my son might have the flu cause I d...
5,happiness,So great to see Oin &amp; Cynthia. So happy. ...


In [6]:
data.groupby(['sentiment']).size()

sentiment
anger          110
boredom        179
enthusiasm     759
fun           1776
happiness     5209
hate          1323
love          3842
relief        1526
sadness       5165
surprise      2187
worry         8459
dtype: int64



---





Due to very small variations between some of emotions & some of error in data :                       (this improves accuracy)

    it's better to combined such data
    
        e.g. 
             anger + boredom + hate = Disgust

In [7]:
# anger + boredom +hate = Disgust (1)
# fun + enthusiasm + surprise = Interest (4)
# relief + happiness + love= Happy (7)
# worry + sadness = Sad(10)


# neutral (REMOVED)


encode = {
    "anger": 1,
    "hate":1, 
    "boredom":1,
    
    "enthusiasm":4,
    "fun":4,
    "surprise":4,
    
    "happiness":7,
    "relief":7,
    "love":7,

    "sadness":10,
    "worry":10,
    
    "neutral":12
}
data["sentiment"] = data["sentiment"].apply(lambda x:encode[x])
data.head()

,sentiment,content
0,7,"@havingmysay dude, that is my favorite sandwi..."
2,1,don't you hate it when you finish all your wor...
3,4,mannnn..... @alllyy_ got an iphone!!! im jealo...
4,10,So i think my son might have the flu cause I d...
5,7,So great to see Oin &amp; Cynthia. So happy. ...




---




importing libraries for preprocessing of data

In [8]:
import re
import nltk
nltk.download('punkt')
nltk.download('stopwords')
# import emoji
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

[nltk_data] Downloading package punkt to /home/shiru/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/shiru/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


here data cleaning & preprocessing is being done

    1. removing web links
    2. replacing emojis (removing emojis is reducing the accuracy)
    3. removing some of sort forms
    4. removing punctuation marks
    5. removing stop words (NO : reducing the accuracy )

In [9]:
def clean_text(df):
    all_reviews = list()
    lines = df["content"].values.tolist()
    for text in lines:
        text = text.lower()
#         text = emoji.demojize(text)
        pattern = re.compile('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
        text = pattern.sub('URL', text)
        
        emoji = re.compile("["
                           u"\U0001F600-\U0001FFFF"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
        text = emoji.sub(r'emoji', text)
        
        text = re.sub(r"i'm", "i am", text)
        text = re.sub(r"he's", "he is", text)
        text = re.sub(r"she's", "she is", text)
        text = re.sub(r"that's", "that is", text)        
        text = re.sub(r"what's", "what is", text)
        text = re.sub(r"where's", "where is", text) 
        text = re.sub(r"\'ll", " will", text)  
        text = re.sub(r"\'ve", " have", text)  
        text = re.sub(r"\'re", " are", text)
        text = re.sub(r"\'d", " would", text)
        text = re.sub(r"\'ve", " have", text)
        text = re.sub(r"won't", "will not", text)
        text = re.sub(r"don't", "do not", text)
        text = re.sub(r"did't", "did not", text)
        text = re.sub(r"can't", "can not", text)
        text = re.sub(r"it's", "it is", text)
        text = re.sub(r"couldn't", "could not", text)
        text = re.sub(r"have't", "have not", text)
        text = re.sub(r'@\w+', 'NAME', text)
        text = re.sub(r"\n't", " not", text)
        
        text = re.sub(r"[,.\"@#$%^&*(){}/;`~:<>+=-]", "", text)
        tokens = word_tokenize(text)
        table = str.maketrans('', '', string.punctuation)
        stripped = [w.translate(table) for w in tokens]
        words = [word for word in stripped if word.isalpha()]
#         
#         stop_words = set(stopwords.words("english"))
#         stop_words.discard("not")
#         words = [w for w in words if not w in stop_words]
        
        words = ' '.join(words)
        all_reviews.append(words)
    return all_reviews

# df = pd.DataFrame(["I'll suggest you to open https://www.google.com/ and search FUN game is on 🔥 🔥"])
# df.columns = ["content"]
# cleaned_text = clean_text(df)
# cleaned_text

all_reviews = clean_text(data)
all_reviews[0:5]

['NAME dude that is my favorite sandwich place ever ummm did you take pictures',
 'do not you hate it when you finish all your work and there s still hours left of work time',
 'mannnn NAME got an iphone im jealous',
 'so i think my son might have the flu cause i def just cleaned up a stanky puke mess poor pumkpin',
 'so great to see oin amp cynthia so happy dinner was great cute little place too bad oin got sick afterwards']

---

it's time to vectorization

In [10]:
# from sklearn.feature_extraction.text import CountVectorizer
# V = CountVectorizer(min_df = 8)   

from sklearn.feature_extraction.text import TfidfVectorizer
V = TfidfVectorizer(min_df = 3)

X = V.fit_transform(all_reviews).toarray()

import pickle
pickle.dump(V,open("tfidf1.pkl","wb"))               # Saving...

y = data["sentiment"]
print(np.shape(X))
print(np.shape(y))

(30535, 7112)
(30535,)


---

***Here onwards different models and their accuracies are mentioned***

## **DecisionTreeClassifier**

In [11]:
# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.22, random_state = 0)

# from sklearn.tree import DecisionTreeClassifier
# model = DecisionTreeClassifier(criterion="entropy", random_state=41)


# model.fit(X_train,y_train)

# y_pred = model.predict(X_test)

# from sklearn.metrics import accuracy_score, f1_score, precision_score

# print(accuracy_score(y_test, y_pred))
# print(f1_score(y_test, y_pred, average='weighted'))
# print(precision_score(y_test, y_pred,average='weighted'))

Outputs : 
          **DecisionTreeClassifier**

            CountVectorizer

                  accuracy_score    :  0.4901756475141411
                  f1_score          :  0.4842023389421267
                  precision_score   :  0.4791195973185888

                  
            TfidfVectorizer

                  accuracy_score    :  0.4778207799940459
                  f1_score          :  0.4720412527120386
                  precision_score   :  0.4670470223722779

## **MultinomialNB**

In [12]:
# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.22, random_state = 0)


# from sklearn.naive_bayes import MultinomialNB
# model = MultinomialNB()

# model.fit(X_train,y_train)

# y_pred = model.predict(X_test)

# from sklearn.metrics import accuracy_score, f1_score, precision_score

# print(accuracy_score(y_test, y_pred))
# print(f1_score(y_test, y_pred, average='weighted'))
# import warnings
# warnings.filterwarnings('ignore')
# print(precision_score(y_test, y_pred,average='weighted'))

Outputs : **MultinomialNB**

        CountVectorizer
 
              accuracy_score    :  0.5913962488835963
              f1_score          :  0.5665193028603359
              precision_score   :  0.5577191344190713
 
 
        TfidfVectorizer
 
              accuracy_score    :  0.6019648704971717
              f1_score          :  0.5314850706524067
              precision_score   :  0.5123836343721758


## **LogisticRegression**

In [13]:
# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.22, random_state = 0)


# from sklearn.linear_model import LogisticRegression 
# model = LogisticRegression(max_iter=1000000)

# model.fit(X_train,y_train)

# y_pred = model.predict(X_test)

# from sklearn.metrics import accuracy_score, f1_score, precision_score

# print(accuracy_score(y_test, y_pred))
# print(f1_score(y_test, y_pred, average='weighted'))
# print(precision_score(y_test, y_pred,average='weighted'))

Outputs : **LogisticRegression**

    CountVectorizer
 
          accuracy_score    :  0.5949687406966359
          f1_score          :  0.5628047013642141
          precision_score   :  0.5530546186998219
 
 
    TfidfVectorizer
 
          accuracy_score    :  0.6122357844596606
          f1_score          :  0.5680690841913283
          precision_score   :  0.5778073438272063


## **SGDClassifier**

In [14]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.22, random_state = 0)


from sklearn.linear_model import SGDClassifier 
model = SGDClassifier()

model.fit(X_train,y_train)

y_pred = model.predict(X_test)

from sklearn.metrics import accuracy_score, f1_score, precision_score

print(accuracy_score(y_test, y_pred))
print(f1_score(y_test, y_pred, average='weighted'))
print(precision_score(y_test, y_pred,average='weighted'))

from joblib import dump, load
dump(model, 'emotion.joblib') 

0.6161059839237868
0.5503842667131983
0.5886657383709684


['emotion.joblib']

Outputs : **SGDClassifier**

    CountVectorizer
    
          accuracy_score    :  0.6064304852634713
          f1_score          :  0.5629850601936482
          precision_score   :  0.5586409784503499
          
    TfidfVectorizer
    
          accuracy_score    :  0.6172968145281333
          f1_score          :  0.5536703949888738
          precision_score   :  0.6039685467932462


---

# **Conclusion :**

Among all above ML models SGDClassifier seems to best for our data 

it gives results upto :

    SGDClassifier
            
          TfidfVectorizer
        
            accuracy_score    :  0.6172968145281333
            f1_score          :  0.5536703949888738
            precision_score   :  0.6039685467932462


---

### ***it's time to check our model:***

In [15]:
content = 'hurreeyyy.!!! i am feeling very happy after completing this project, I loved it'


from joblib import dump, load
import numpy as np
model = load('emotion.joblib') 

import pickle
V =  pickle.load(open("tfidf1.pkl","rb"))

features = V.transform([content]).toarray()

mood = model.predict(features)[0]

# anger + boredom +hate = Disgust (1)
# fun + Enthu + surprise = Interest (4)
# relief + happiness + love= Happy (7)
# worry + sadness = Sad(10)


if (mood == 1):
    print("Disgust : anger / boredom / hate..." )
elif(mood == 4):
    print("Interest : surprise / enthusiasm / fun")
elif(mood == 7):
    print("Happy : happiness / love / relief")
else:
    print("Sad : sadness / worry")
    

Happy : happiness / love / relief




---


                                                           --- The End ---


---



